# Variational Autoencoder
Based on variational inference

$$P(Z|X) = \frac{P(Z,X)}{P(X)}$$

## Some basic information theory
### Information 
$I = -log(P(x))$
* Measures the factor of uncertainty removed when x is known
* 1 bit can be thought of as information that reduces uncertainty
    by a factor of 2
E.g. Let's say there is 50% chance of weather being sunny and 
50% chance of rain tomorrow. When the weather station tells us it is going to be sunny, they have given us 1 bit of information.

> Uncertainty reduction is the inverse of the event's probability

E.g. If the weather probabilities are sunny 75% and rain 25%,
finding out that it is going to be rainy will reduce our 
uncertainty by $1/0.25 = 4$. This corresponds to $log_2(4) = 2$ bits of information $log_2(\frac{1}{0.25}) = -log_2(0.25)$

### Entropy 
$H = \sum_x-log(P(x)) * P(x))$  
* Can be thought of as average amount of information relayed by a certain distribution
E.g in the above case, the weather station on average transmits
$$ H = 0.75 \times -log(0.75) + 0.25 \times (-log(0.25)) 
  = 0.81$$ bits of useful information

### Cross-entropy
$$H(P,Q) = \sum_x P(x)(-log(Q(x))$$

E.g Lets say we use 2 bits to encode our weather prediction
this can be thought of as us predicting the weather to have
a 25% chance of either being sunny or rainy
The average number of actual bits sent is
$H = 0.75 \times 2 + 0.25 \times 2 = 2 $bits. If using different number of bits for the different predictions $H = 0.75 \times 2 + 0.25 \times 3 = 2.25 $bits 

Based on the entropy and cross-entropy, we can see that our _predicted_ probability distribution Q(x) differs from the _actual_ probability distribution P(x) by 
$KL(P||Q) = 2.25 - 0.81 = 1.54 $bits  
If predictions are perfect i.e. Q(x) = P(x), then H(P,Q) = H(P)  
Therefore, $H(P,Q) = H(P) + KL(P||Q)$  
$KL(P||Q)$ means KL-divergence of Q w.r.t P

\begin{align}
KL(P||Q) &= H(P,Q) - H(P)\\
         &= \sum_x P(x)(-log(Q(x)) -  \sum_xP(x)(-log(P(x))\\
         &= \sum_{x} P(x)(-log(Q(x) - (-log(P(x)))\\
         &= \sum_x P(x)(log(P(x)) - log(Q(x)))\\
         &= \sum_x P(x)(log(\frac{P(x)}{Q(x)}))\\
\end{align}
#### Some properties of KL-divergence
1. $KL(P||Q)$ is alwaysgreater than or equal to 0
2. $KL(P||Q)$ is not the same as $KL(Q||P)$


## Variational Bayes

$$P(Z|X) = \frac{P(Z,X)}{P(X)} = \frac{P(X|Z)P(Z)}{P(X)}$$

We don't know P(X). If we were to compute it,
$P(X) = \int{P(X|Z)P(Z)dZ}$
* Intractable in many cases
* If distributions are high dimensional, integral is multi-integral

Thus, we can try to approximate the distribution. One method to approximate is Monte Carlo method (Gibbs sampling and other sampling methods) which is unbiased with high variance.

Another is variational inference which has low variance but is biased

1. Approximate P(Z|X) with Q(Z) that is tractable e.g. Gaussian
2. Play with the parameters of Q(Z) in a way that it gets close enough to P(Z|X) i.e. minimize $KL(Q(Z)||P(Z|X))$

This brings us to the following objective of minimizing 


\begin{align}
KL(Q(Z)||P(Z|X)) &= \sum_z Q(Z)log(\frac{Q(Z)}{P(Z|X)})\\
                 &= - \sum_zQ(Z) log(\frac{P(Z|X)}{Q(Z)})\\
                 &= - \sum_z Q(Z) log(\frac{P(X,Z)}{P(X) Q(Z)})\\
                 &= - \sum_z Q(Z) (log(\frac{P(X,Z)}{Q(Z)}) - log(P(X)))\\
                 &= - \sum_z Q(Z) log(\frac{P(X,Z)}{Q(Z)}) + log(P(X))\\
\end{align}
\begin{align}
\therefore log(P(X))    &= KL(Q(Z)||P(Z|X)) + \sum_z Q(Z)log(\frac{P(X,Z)}{Q(Z)})\\
                 &= KL(Q(Z)||P(Z|X)) + L\\
\end{align}


As $log(P(X))$ is a constant, to minimize $KL(Q(Z)||P(Z|X))$,
we just need to maximize $L$.

$$\because KL(Q(Z)||P(Z|X)) \geq 0$$,  
$$L \leq P(X)$$ Thus, L is a lower bound of P(X).

\begin{align}
L &= \sum_z Q(Z) log(\frac{P(X,Z)}{Q(Z)})\\
  &= \sum_z Q(Z) log(\frac{P(X|Z)P(Z)}{Q(Z)})\\
  &= \sum_z Q(Z)(log(P(X|Z)) + log(\frac{P(Z)}{Q(Z)}))\\
  &= \sum_z Q(Z) log(P(X|Z))) + \sum_z Q(Z) log(\frac{P(Z)}{Q(Z)})\\
\end{align}

$$\sum_z Q(Z) log(P(X|Z))) = E_{Q(Z)}P(X|Z)$$
$$\sum_z Q(Z) log(\frac{P(Z)}{Q(Z)}) = -KL(Q(Z)||P(Z))$$
Representing L as an autoencoder

X --> Q(Z|X) --> Z --> P(X|Z) --> X'

$E_{Q(Z)}P(X|Z)$ term acts as reconstruction error.
$P(X|Z)$ is deterministic meaning one input will get the same output all the time. Thus, it can be considered $P(X|X')$.

If $P(X|X')$ is gaussian
$$P(X|X') = e^{-|X - X'|^2}$$
$$log(P(X|X')) = -|X - X'|^2$$ --> L2 loss

If Bernoulli distribution, will be similar to cross-entropy

So far the network is all deterministic
> To make it probabilistic, 
    encoder should not parametrize Z but instead the parametrize
    the distribution that generates Z i.e. $\mu$ and $\sigma$



In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import traceback

In [ ]:
df = pd.read_csv('../train.csv')
labels = np.asarray(df.label).reshape(42000,1)
train_x = np.asarray(df[df.columns[1:]]).reshape(42000,28*28)/255.0

In [ ]:
############################ SIMPLE MODEL ###############################################
mu_data, sigma_data = 3., 1.75
mu_z, sigma_z = 0., 1.
alpha = 0.2
np.random.seed(0)
#data_distribution = np.random.normal(mu_data, sigma_data, (40000, 10))
#data_distribution /= np.max(data_distribution,axis=1).reshape(40000,1)

latent_distribution = np.random.normal(mu_z, sigma_z, (42000, 16))
tf.Graph().as_default()
X = tf.placeholder(tf.float32, (None, 784))
with tf.name_scope('encoder'):
    en_1 = tf.layers.BatchNormalization()(X)
    en_1 = tf.layers.Dense(512)(en_1)
    en_1 = tf.maximum(en_1, alpha*en_1)
  #  en_2 = tf.layers.Dense(256)(en_1)
  #  en_2 = tf.maximum(en_2, alpha*en_2)
  #  en_3 = tf.layers.Dense(128)(en_2)
  #  en_3 = tf.maximum(en_3, alpha*en_3)
  #  encoded = tf.layers.Dense(32)(en_3)
    #encoded = tf.maximum(encoded, alpha*encoded)
    mean = tf.layers.Dense(16)(en_1)
    log_stdev = tf.layers.Dense(16)(en_1)
#log_stdev = encoded[:,0:16]
#mean = encoded[:,16:32]
latent_samples = tf.placeholder(tf.float32, (None,16))
Z = latent_samples * tf.exp(log_stdev) + mean
with tf.name_scope('decoder'):
    de_3 = tf.layers.BatchNormalization()(Z)
 #   de_1 = tf.layers.Dense(128)(Z)
 #   de_1 = tf.maximum(de_1, alpha*de_1)
 #   de_2 = tf.layers.Dense(256)(de_1)
 #   de_2 = tf.maximum(de_2, alpha*de_2)
    de_3 = tf.layers.Dense(512)(de_3)
    de_3 = tf.maximum(de_3, alpha*de_3)
    output = tf.layers.Dense(784,activation=tf.nn.sigmoid)(de_3)
scale = 1.
beta = 1.3
recon_loss = tf.reduce_sum((tf.square(X - output)),axis=1)#tf.keras.losses.binary_crossentropy(X,output) #
kl_loss = - 0.5 * tf.reduce_sum(1 + log_stdev - tf.square(mean) - tf.square(tf.exp(log_stdev)), axis=1)
loss = tf.reduce_mean(scale*recon_loss + beta*kl_loss)
adam = tf.train.AdamOptimizer()
train = adam.minimize(loss)

In [ ]:
epochs = 100
bs = 100
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess,'./checkpoints/simple.ckpt')
    try:
        for i in range(epochs):
            print('Ep: ' + str(i))
            #plt.imshow(sess.run(output,{X:train_x[0:1],latent_samples:latent_distribution[0:1]})[0].reshape((28,28)))
            for b in range(420):
            #batch = train_x[b*batch_s:]
                sess.run(train,{X:train_x[b*bs:b*bs+1],latent_samples:latent_distribution[b*bs:b*bs+1]})
                if b % 1 == 0:
                    saver.save(sess,'./checkpoints/simple.ckpt')
                    print("Loss: " + str(sess.run(loss,{X:train_x[b*bs:b*bs+1],latent_samples:latent_distribution[b*bs:b*bs+1]})))
    except Exception as e:
        print(e)
    finally:
        saver.save(sess,'./checkpoints/simple.ckpt')
        print('Model saved')
        
    

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'./checkpoints/simple.ckpt')
    #print(data_distribution[0:5])
    #print(sess.run(output,{X:data_distribution,latent_samples:latent_distribution})[0:5])
    #print(sess.run(loss,{X:data_distribution,latent_samples:latent_distribution}))
    plt.imshow(sess.run(output,{X:train_x[1].reshape([-1,784]),latent_samples:latent_distribution[1].reshape([-1,16])})[0].reshape((28,28)))
    

In [ ]:
plt.imshow(train_x[2].reshape(28,28))

In [ ]:
####################### MNIST MODEL ########################################
k = 9
mu_z, sigma_z = 0., 1.
latent_dist = np.random.normal(mu_z,sigma_z,(42000,7,7,32))
tf.Graph().as_default()
X = tf.placeholder(tf.float32, (None, 28, 28, 1))
with tf.name_scope('encoder'):
    en1 = tf.layers.Conv2D(64, (k,k))(X)
    en1 = tf.nn.leaky_relu(en1)
    en2 = tf.layers.Conv2D(64, (k,k), strides=(2,2))(en1)
    en2 = tf.nn.leaky_relu(en2)
    en3 = tf.layers.Conv2D(64, (3,3), strides=(1,1))(en2)
    en3 = tf.nn.leaky_relu(en3)
    en4 = tf.layers.Conv2D(64, (3,3), strides=(1,1))(en3)
    en4 = tf.nn.leaky_relu(en3)
    encoded = tf.layers.Conv2D(32, (3,3),padding='same',strides=(2,2))(en2)
    #encoded = tf.nn.leaky_relu(encoded)
batch_size = tf.shape(encoded)[0]
latent_samples = tf.placeholder(tf.float32,(None,7,7,32))
log_sigma = encoded[:,:,:,0:32]
mu = encoded[:,:,:,32:64]
Z = tf.exp(log_sigma) * latent_samples + mu 

with tf.name_scope('decoder'):
    de_3 = tf.layers.Dense(512)(Z)
    de_3 = tf.maximum(de_3, alpha*de_3)
    output = tf.layers.Dense(784,activation=tf.nn.sigmoid)(de_3)
'''
with tf.name_scope('decoder'):
    de1 = tf.layers.Conv2DTranspose(64,(3,3),padding='same',strides=(2,2))(Z)
    de1 = tf.nn.leaky_relu(de1)
    de2 = tf.layers.Conv2DTranspose(64,(7,7))(de1)
    de2 = tf.nn.leaky_relu(de2)
    de3 = tf.layers.Conv2DTranspose(64,(9,9))(de2)
    de3 = tf.nn.leaky_relu(de3)
    output = tf.layers.Conv2DTranspose(1,(3,3),padding='same',activation=tf.nn.sigmoid)(de3)
'''
beta = 1.3
squared_loss = tf.reduce_sum(tf.)#tf.reduce_sum(tf.square(X - output))
kl = beta * tf.reduce_sum(tf.square(tf.exp(log_sigma)) + tf.square(mu) - 1. - log_sigma)
mnist_train = tf.train.AdamOptimizer().minimize(squared_loss+kl)

In [ ]:
epochs = 10
batch_s = 1
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess,'./checkpoints/simple.ckpt')
    try:
        for i in range(epochs):
            print('Epoch ' + str(i))
            for b in range(42000):
                sess.run(mnist_train,{X:train_x[b*batch_s:b*batch_s + batch_s],latent_samples:latent_dist[b*batch_s:b*batch_s + batch_s]})
                if b % 100 == 0:
                    saver.save(sess,'./checkpoints/mnist.ckpt')
                    print("Loss: " + str(sess.run(squared_loss,{X:train_x[b*batch_s:b*batch_s + batch_s],latent_samples:latent_dist[b*batch_s:b*batch_s + batch_s]})))
    except Exception as e:
        traceback.format_exc(e)
    finally:
        saver.save(sess,'./checkpoints/mnist.ckpt')
        print('Model saved')
